In [ ]:
# ============================================================
# PYTHON GROSS PREMIUM CALCULATION
# ============================================================

from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import numpy as np

def parse_assumption(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)

    x = str(x).strip()
    if x == "":
        return None

    if x.isalpha():
        return x

    if "%" in x:
        return float(x.replace("%", "").replace(",", "")) / 100

    if "RM" in x:
        return float(x.replace("RM", "").replace(",", ""))

    return float(x.replace(",", ""))

def get_param(*keys, default=0):
    for k in keys:
        if k in params:
            return params[k]
    return default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet = gc.open('Mortality_Database')
print("✅ Connected to database")

sheet_mortality = spreadsheet.get_worksheet(0)
rows = sheet_mortality.get_all_values()

df = pd.DataFrame(rows[1:], columns=rows[0])
df.columns = df.columns.str.strip()

for c in ['age', 'qx_base', 'qx_shocked']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df = df.set_index('age').sort_index()
print("✅ Mortality table loaded")

sheet_assump = spreadsheet.worksheet('Assumptions')
rows = sheet_assump.get_all_values()

params = {}
for r in rows[1:]:
    params[r[0].strip().lower()] = parse_assumption(r[1])

print("✅ Assumptions loaded")

faceamount = params['sum assured']
intrate    = params['interest rate']
mortload   = params['mortality loading']

issueExpenses       = params['issue expense']
maintenanceExpenses = params['maintenance expense']

firstyearcomm = get_param(
    'first year commission',
    'first year commision',
    default=0
)

renewalyearcomm = get_param(
    'renewal commission',
    'renewal commision',
    default=0
)

profit = params['profit margin']

basis_text = params['mortality basis']
TERM = int(params['term years'])

v = 1 / (1 + intrate)

qx_col = 'qx_base' if str(basis_text).lower() == 'base' else 'qx_shocked'

valid_qx = df[qx_col].dropna()
STARTQX = int(valid_qx.index.min())
ENDQX   = int(valid_qx.index.max())

QX, lx, dx = {}, {}, {}

for age in range(STARTQX, ENDQX + 1):
    QX[age] = df.loc[age, qx_col] * mortload

lx[STARTQX] = 10000

for age in range(STARTQX, ENDQX):
    lx[age + 1] = lx[age] * (1 - QX[age])
    dx[age] = lx[age] - lx[age + 1]

def calculate_gp(age):

    if age < STARTQX or age + TERM > ENDQX:
        return np.nan

    sumbenefit = 0.0
    sumannuity = 0.0
    sumexpenses = 0.0
    sumpremiumexpenses = 0.0

    for n in range(TERM):

        if n == 0:
            commission = firstyearcomm
            policyExp  = issueExpenses
        else:
            commission = renewalyearcomm
            policyExp  = maintenanceExpenses

        sumbenefit += (v ** (n + 1)) * dx[age + n] / lx[age]
        sumannuity += (v ** n) * lx[age + n] / lx[age]
        sumexpenses += policyExp * (v ** n) * lx[age + n] / lx[age]
        sumpremiumexpenses += (v ** n) * lx[age + n] / lx[age] * (commission + profit)

    return (sumbenefit * faceamount + sumexpenses) / (sumannuity - sumpremiumexpenses)

print("\n" + "="*65)
print(" FUTURESECURE – PYTHON GROSS PREMIUM RESULTS")
print("="*65)

print("INPUT SUMMARY")
print("-"*65)
print(f"Mortality Basis        : {basis_text}")
print(f"Policy Term            : {TERM} years")
print(f"Interest Rate          : {intrate*100:.2f}%")
print(f"Sum Assured            : RM {faceamount:,.2f}")
print(f"Mortality Loading      : {mortload}")
print(f"Issue Expense          : RM {issueExpenses:,.2f}")
print(f"Maintenance Expense    : RM {maintenanceExpenses:,.2f}")
print(f"First Year Commission  : {firstyearcomm*100:.2f}%")
print(f"Renewal Commission     : {renewalyearcomm*100:.2f}%")
print(f"Profit Margin          : {profit*100:.2f}%")

print("-"*65)

print("GROSS PREMIUM RESULTS")
print("-"*65)
for age in [30, 40, 50]:
    gp = calculate_gp(age)
    print(f"Age {age} → Gross Premium : RM {gp:,.2f}")

print("="*65)

✅ Connected to database
✅ Mortality table loaded
✅ Assumptions loaded

 FUTURESECURE – PYTHON GROSS PREMIUM RESULTS
INPUT SUMMARY
-----------------------------------------------------------------
Mortality Basis        : Shocked
Policy Term            : 10 years
Interest Rate          : 4.00%
Sum Assured            : RM 1,500,000.00
Mortality Loading      : 0.06
Issue Expense          : RM 400.00
Maintenance Expense    : RM 100.00
First Year Commission  : 3.00%
Renewal Commission     : 3.00%
Profit Margin          : 2.00%
-----------------------------------------------------------------
GROSS PREMIUM RESULTS
-----------------------------------------------------------------
Age 30 → Gross Premium : RM 360.69
Age 40 → Gross Premium : RM 651.17
Age 50 → Gross Premium : RM 1,477.91
